In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
# @IMPORT-MERGE
import os
os.environ["MPLCONFIGDIR"] = "/tmp"
import numpy as np
import pandas as pd
import itertools
import cv2
import random
from IPython.display import HTML, display
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run.sigproc_v2.sigproc_v2_result import df_filter, radmat_from_df_filter
from plaster.run.plots import plots, plots_dev
from plaster.run.plots.plots_sigproc import plot_psfs
from plaster.run.sigproc_v2.synth import Synth
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.utils import data
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.utils import data
from plaster.tools.calibration.calibration import Calibration
from plaster.tools.ipynb_helpers.displays import hd, movie
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/abbe8_2t")
run = job.runs[0]

# Calibration

In [ ]:
# Show Calibration information
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    psfs = run.sigproc_v2.params.calibration.psfs(ch_i=ch_i)

    with z(_noaxes=True):
        display("Below:")
        with z(_cols=6):
            n_zs = psfs.shape[0]
            for z_i in range(0, n_zs // 2):
                plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z)

        display("Most in Focus:")
        z_i = n_zs // 2
        plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z, _noaxes=True, _notools=True)

        display("Above:")
        with z(_cols=6):
            for z_i in range(n_zs // 2 + 1, n_zs):
                plot_psfs(psfs[z_i], scale=2.0, f_title=f"ch_i={ch_i}, z_i={z_i}", _zplots_context=z)
                
    # Show the Calibration illumination balance
    illum = np.array(run.sigproc_v2.params.calibration[f"regional_illumination_balance.instrument_channel[{ch_i}]"])
    z.im(1.0 / illum, f_title="Illumination map")

# Quality

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    qdf = run.ims_import.qualities()
    quality = qdf[qdf.channel_i == ch_i].sort_values(["quality"])
    z.hist(quality.quality, _size_x=800, _size_y=150, f_title=f"Quality distribution channel {ch_i}")

    row_iz = utils.ispace(0, len(qdf), 3)

    # COMBINE all images for common percentile calculations
    ims = np.concatenate([
        run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i].flatten()
        for row in qdf.iloc[row_iz].itertuples()
    ])
    bot, top = np.percentile(ims, (50, 99.99))

    # SHOW example of worst, median, and best all using the same cspan
    hd("h3", f"Examples of frames by quality")
    with z(_cols=3, _cspan=(bot, top)):
        names = ("worst", "median", "best")
        for name, row in zip(names, qdf.iloc[row_iz].itertuples()):
            z.im(run.sigproc_v2.aln_ims[row.field_i, row.channel_i, row.cycle_i], f_title=f"Channel: {ch_i} {name}")            

# Alignment

In [ ]:
field_df = run.sigproc_v2.fields().copy()
field_df["alignment"] = np.sqrt(field_df.aln_x**2 + field_df.aln_y**2)
alignment = field_df.groupby("field_i").alignment.max().values
z.cols(alignment, f_x_axis_label="field_i", f_y_axis_label="n_pixels", f_title="Max. alignment dist.")
good_field_iz = np.argwhere(alignment < 50)

# SNR

In [ ]:
df = run.sigproc_v2.fields__n_peaks__peaks__radmat()

In [ ]:
from plaster.tools.image.coord import ROI, YX, HW

def df_filter(
    df,
    fields=None,
    reject_fields=None,
    roi=None,
    channel_i=0,
    dark=None,
    on_through_cy_i=None,
    off_at_cy_i=None,
    monotonic=None,
    min_intensity_cy_0=None,
    max_intensity_cy_0=None,
    max_intensity_any_cycle=None,
    min_intensity_per_cycle=None,
    max_intensity_per_cycle=None,
    max_aspect_ratio=None,
    radmat_field="signal",
    **kwargs,
):
    """
    A general filtering tool that operates on the dataframe returned by
    sigproc_v2.fields__n_peaks__peaks__radmat()
    """
    n_channels = df.channel_i.max() + 1
    n_cycles = df.cycle_i.max() + 1

    # REMOVE unwanted fields
    if fields is None:
        fields = list(range(df.field_i.max() + 1))
    if reject_fields is not None:
        fields = list(filter(lambda x: x not in reject_fields, fields))
    _df = df[df.field_i.isin(fields)].reset_index(drop=True)
    
    # REMOVE unwanted peaks by ROI
    if roi is None:
        roi = ROI(YX(0, 0), HW(df.raw_y.max(), df.raw_x.max()))
    _df = _df[
        (roi[0].start <= _df.raw_y)
        & (_df.raw_y < roi[0].stop)
        & (roi[1].start <= _df.raw_x)
        & (_df.raw_x < roi[1].stop)
    ].reset_index(drop=True)

    # OPERATE on radmat if needed
    fields_that_operate_on_radmat = [
        dark,
        on_through_cy_i,
        off_at_cy_i,
        monotonic,
        min_intensity_cy_0,
        max_intensity_cy_0,
        max_intensity_any_cycle,
        min_intensity_per_cycle,
        max_intensity_per_cycle,
    ]

    if any([field is not None for field in fields_that_operate_on_radmat]):
        assert 0 <= channel_i < n_channels

        rad_pt = pd.pivot_table(
            _df, values="signal", index=["field_i", "peak_i"], columns=["channel_i", "cycle_i"]
        )
        
#         if max_aspect_ratio is not None:
#             asr_pt = (
#                 pd.pivot_table(
#                     _df, values="aspect_ratio", index=["field_i", "peak_i"], columns=["channel_i", "cycle_i"]
#                 )
#             )
# NOT SURE HOW TO APPLY THE DARK MASK
#             mask = asr_pt.loc[:, channel_i] <= max_aspect_ratio
#             asr[radmat < dark] = np.nan
#             with warnings.catch_warnings():
#                 warnings.simplefilter("ignore", category=RuntimeWarning)
#                 keep_peaks_mask &= np.nanmean(asr, axis=1) <= max_aspect_ratio

        if on_through_cy_i is not None:
            assert dark is not None
            rad_pt
            keep_peaks_mask &= np.all(radmat[:, 0 : on_through_cy_i + 1] > dark, axis=1)

        if off_at_cy_i is not None:
            assert dark is not None
            keep_peaks_mask &= np.all(radmat[:, off_at_cy_i:] < dark, axis=1)

        if monotonic is not None:
            d = np.diff(radmat, axis=1)
            keep_peaks_mask &= np.all(d < monotonic, axis=1)

        if min_intensity_cy_0 is not None:
            keep_peaks_mask &= radmat[:, 0] >= min_intensity_cy_0

        if max_intensity_cy_0 is not None:
            keep_peaks_mask &= radmat[:, 0] <= max_intensity_cy_0

        if max_intensity_any_cycle is not None:
            keep_peaks_mask &= np.all(radmat[:, :] <= max_intensity_any_cycle, axis=1)

        if min_intensity_per_cycle is not None:
            for cy_i, inten in enumerate(min_intensity_per_cycle):
                if inten is not None:
                    keep_peaks_mask &= radmat[:, cy_i] >= inten

        if max_intensity_per_cycle is not None:
            for cy_i, inten in enumerate(max_intensity_per_cycle):
                if inten is not None:
                    keep_peaks_mask &= radmat[:, cy_i] <= inten

        # Apply the peak mask back into the dataframe
        # REMEMBER: radmat is the radmat for the SMALLER, possibly filtered _df
        # so the rows of this mat are not peak_iz in the original df
        peak_iz = np.arange(radmat.shape[0])
        _df = df[df.peak_i.isin(peak_iz[keep_mask])]

    return _df


def df_to_radmat(df, n_channels, n_cycles, radmat_field="signal"):
    """
    Convert the dataframe filtered by df_filter into a radmat
    """
    df["chcy_i"] = df.channel_i * n_cycles + df.cycle_i
    radmat = (
        pd.pivot_table(
            df, values=radmat_field, index=["field_i", "peak_i"], columns=["chcy_i"]
        )
        .reset_index()
        .rename_axis(None, axis=1)
        .drop(columns=["field_i", "peak_i"])
        .reindex(np.arange(n_channels * n_cycles), axis=1, fill_value=np.nan)
    ).values
    return np.nan_to_num(radmat.reshape(radmat.shape[0], n_channels, n_cycles))


def radmat_from_df_filter(df, radmat_field="signal", **kwargs):
    """
    Apply the filter args from df_filter and return a radmat
    """
    n_channels = df.channel_i.max() + 1
    n_cycles = df.cycle_i.max() + 1
    _df = df_filter(df, radmat_field=radmat_field, **kwargs)
    return df_to_radmat(_df, n_channels=n_channels, n_cycles=n_cycles, radmat_field=radmat_field)


In [ ]:
_df = df_filter(df, fields=good_field_iz)

In [ ]:
(_df.field_i == 89).sum()

In [ ]:
# def df_to_radmat(df, n_channels, n_cycles, radmat_field="signal"):
#     """
#     Convert the dataframe filtered by df_filter into a radmat (n_peaks, n_channels, n_cycles)
#     """
#     df["chcy_i"] = df.channel_i * n_cycles + df.cycle_i
#     radmat = (
#         pd.pivot_table(
#             df, values=radmat_field, index=["field_i", "peak_i"], columns=["chcy_i"]
#         )
#         .reset_index()
#         .rename_axis(None, axis=1)
#         .drop(columns=["field_i", "peak_i"])
#         .reindex(np.arange(n_channels * n_cycles), axis=1, fill_value=np.nan)
#     ).values
#     return np.nan_to_num(radmat.reshape(radmat.shape[0], n_channels, n_cycles))

df = pd.DataFrame(dict(
    field_i  =[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
    channel_i=[0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
    cycle_i  =[0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2],
    peak_i   =[2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5],
    signal   =[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
))
# display(df)

# Assume peak_i is global unique
n_cycles = 3
pt = pd.pivot_table(
    df, values="signal", index=["peak_i"], columns=["channel_i", "cycle_i"]
)
# display(pt)

ch0_pt = pt.loc[:, 0]
display(ch0_pt)

mask = ch0_pt.loc[:, 2] > 3
mask &= ch0_pt.loc[:, 0] < 13

ch0_pt[mask].index.values

# peaks_on_at_end = pt.loc[:, (0, 2)] > 3
# display(peaks_on_at_end)
# pt.peak_i#loc[(None, peaks_on_at_end)]

#display(pt.loc[(1, (2, 4)), 0])
#mask = pt.loc[:, 1] >= 3
# m = np.all(mask, axis=1)
# display(m)
# pt.loc[m, 0]

# rad = df_to_radmat(df, 2, 3, radmat_field="signal")
# rad[:, 0]

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")
    snr = radmat_from_df_filter(_df, radmat_field="snr")
    snr = snr[:, ch_i, :]
#     _df = df.groupby(["field_i", "cycle_i"]).snr.mean().reset_index()
#     snr_by_field_cycle = (
#         pd.pivot_table(
#             _df, values="snr", index="field_i", columns=["cycle_i"]
#         )
#         .reset_index()
#         .rename_axis(None, axis=1)
#         .drop(columns="field_i")
#     ).values

    top = np.percentile(snr, 97)

    with z(_cols=2):
        z.hist(
            snr, _bins=(0, top, 200),
            f_y_axis_label="count", f_x_axis_label="SNR",
            f_title=f"SNR distribution ch_i={ch_i}"
        )
        z.im_clus(
            snr_by_field_cycle,
            f_y_axis_label="field_i", f_x_axis_label="cycle_i",
            f_title=f"Mean SNR by field and cycle ch_i={ch_i}"
        )
    


# Signal

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    sig = run.sigproc_v2.sig()[:, ch_i, :]
    _df = df.groupby(["field_i", "cycle_i"]).signal.mean().reset_index()
    sig_by_field_cycle = (
        pd.pivot_table(
            _df, values="signal", index="field_i", columns=["cycle_i"]
        )
        .reset_index()
        .rename_axis(None, axis=1)
        .drop(columns="field_i")
    ).values

    top = np.percentile(sig, 97)

    with z(_cols=2):
        z.hist(
            sig, _bins=(0, top, 200),
            f_y_axis_label="count", f_x_axis_label="Signal",
            f_title=f"Signal distribution ch_i={ch_i}"
        )
        z.im_clus(
            sig_by_field_cycle,
            f_y_axis_label="field_i", f_x_axis_label="cycle_i",
            f_title=f"Mean Sig by field and cycle ch_i={ch_i}",
            _cspan=(0, top),
        )

    z.im_clus(sig, _cspan=(0, top), f_title=f"radmat sample ch_i={ch_i}")

In [ ]:
# Histograms by cycle
for ch_i in range(run.ims_import.n_channels):
    hd("h2", f"Channel {ch_i}")

    sig = run.sigproc_v2.sig()[:, ch_i, :]
    center = np.median(sig[:, 0])
    n_cycles = run.sigproc_v2.n_cycles
    max_x = np.percentile(sig, 99)
    bins = np.linspace(-1000, max_x, 200)
    _hist, _ = np.histogram(sig[:, 0], bins=bins)
    max_y = np.max(_hist)
    with z(_cols=4, _size=200, _noaxes=True, _bins=bins, _range=(0, max_x, 0, max_y*1.2)):
        for cy_i in range(n_cycles):
            _sig = sig[:, cy_i].flatten()
            with z(_merge=True, f_title=f"cy_i={cy_i}"):
                z.hist(_sig)
                z.line(x=[center, center], y=[0, max_y], color="red")
                z.line(x=[0, 0], y=[0, max_y], color="black")


# Image Visualization

In [ ]:
field_channel = (0, 0)

ims = run.ims_import.ims[field_channel[0], field_channel[1], :]
_cper=(20, 99.99)
movie(ims, _cper=_cper, _labels=[
    f"raw fl_i:{field_channel[0]} ch_i:{field_channel[1]} cy_i: {cy_i}"
    for cy_i in range(ims.shape[0])
])

ims = run.sigproc_v2.aln_ims[0, 0, :]
movie(ims, _cper=_cper, _labels=[
    f"aligned & balanced fl_i:{field_channel[0]} ch_i:{field_channel[1]} cy_i: {cy_i}"
    for cy_i in range(ims.shape[0])
])

# TODO: Make a wrapper on movie that is in plots that
# Will wrap circles around it like I do in other places
# so I can see things that pass various criteria

In [ ]:
# View a field, channel, cycle with roll-over properties for debugging

# Change this line to select a different (field, channel, cycle) to view:
field_channel_cycle = (0, 0, 0)

mea = run.ims_import.dim
plots_dev.sigproc_v2_im(run, *field_channel_cycle, _size=mea, _range=(0, mea, 0, mea))


# Wizards

In [ ]:
plots.wizard_scat_df(run, channel_i=0)

In [ ]:
plots.wizard_xy_df(run, channel_i=0)

In [ ]:
plots.wizard_raw_images(run, show_circles=False, peak_i_square=True, square_radius=7)